<a href="https://colab.research.google.com/github/higherbar-ai/open-chat-studio-sim/blob/main/src/simulate-conversations.ipynb" target="_parent"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

# Simulate conversations

This notebook simulates OCS conversations by using one bot to simulate users interacting with another bot.

## Before you run this notebook: set up a user simulator experiment

You will need an experiment configured to simulate user interactions from the user side. This experiment should be set up to expect a persona description and any other context in its first user message, respond with an initial user message to begin the conversation, and then carry on to simulate that persona in all subsequent messages until finally sending a one-word response of "END" to end the simulated conversation. For example, here is the prompt text for an example experiment named `U.S. Health Advice - User Simulator`:

    You are a helpful user simulator for testing a U.S. Health Advice AI assistant. 
    
    The first message you receive from the user will describe the kind of user you should simulate. For example, you might be told that you are to simulate a 42-year-old male with daily headaches that started two weeks ago and have slowly worsened. Your response to that first message should be an initial message to send to the AI assistant, with no prefix or suffix; your response will be sent directly to the AI assistant to begin the conversation. Do your best to simulate what a real humans with the described persona and context would say to begin a conversation in this context. 
    
    All subsequent messages you receive from the user will be messages from a U.S. Health Advice AI assistant, beginning with its response to your initial message. You should respond to each of these messages "in character," acting as the persona described in the first user message. While you should use the persona description to guide your interactions with the AI assistant, it will be far from complete; feel free to make up additional details to fill in gaps in the persona, symptoms, etc. as necessary to plausibly simulate a human user interacting with a health advice assistant. 

    Continue to ask and answer questions until you expect a normal human with the described persona would end the conversation. To end the conversation after receiving any AI message, respond to any AI message with one word alone (with nothing before or after it):

    END

    To be clear, never add "END" to the end of a message for the assistant. I.e., the assistant will always have the last word, in response to which you can use END (alone) to end the conversation.

    Finally: do not continue conversations for more time than a regular human with the described persona would, and under no circumstances should you continue beyond 50 back-and-forth interactions between you and the AI assistant.
    
You can customize your simulator prompt text to suit your needs, but it should always instruct the AI to:
 
1. Expect persona and context details in user message #1 
2. Respond with an initial message to begin the conversation
3. Respond to subsequent user messages as the persona described in the first user message
4. End the conversation with a one-word response of "END" 

## Running in Google Colab

Before running this notebook, you'll need to configure a series of secrets in Google Colab; click the key button in the left sidebar, and be sure to click the toggle to give this notebook access to each of the secrets. These are the secrets used by this notebook:

- `OCS_API_KEY`: your Open Chat Studio API key
- `ATHINA_API_KEY`: your Athina API key (optional; only if you want to export results to Athina)
- `USER_SIMULATOR_EXPERIMENT_ID`: the ID of the user simulator experiment you set up (described above)
- `EXPERIMENT_ID`: the ID of the experiment you want to simulate user interactions with
- `PARTICIPANT_ID`: the participant ID to use for the queries

## Running in a local environment

When you first run the first code cell in this notebook, it will output a template configuration file for you. Edit that file to specify your configuration parameters (see above for their descriptions). 

## Selecting or uploading your simulations to run

The second code cell will prompt you to select or upload a .csv file with the simulations you want to run. This file should have the following columns:

- `context`: a description of the context to simulate, which will be sent to the user simulator experiment as the first message (should include everything needed to simulate a user, including background information, why the user is coming to interact with the bot, etc.)
- `simulation_id`: (optional) a unique identifier for the simulation (if not provided, row number will be used)

## Where results go

The results of the queries will be saved to a file called `simulation_results.csv`. If you're running in Google Colab, click the folder button in the sidebar to view and download that file. If you're running locally, it will be output to the `ocs` subdirectory off of your local directory. 

If an Athina API key is configured, the results will also be exported to an Athina dataset.

In [ ]:
# First, we need to set up our environment handler
try:
    # Try importing directly (local environment)
    from colab_or_local_env import ColabOrLocalEnv  # type: ignore[import]
except ImportError:
    # If import fails, we're probably in Colab, so fetch the file
    import requests
    
    # Fetch the environment handler
    url = "https://raw.githubusercontent.com/higherbar-ai/open-chat-studio-sim/main/src/colab_or_local_env.py"
    response = requests.get(url)
    response.raise_for_status()
    
    # Save it to the current directory
    with open("colab_or_local_env.py", "w") as f:
        f.write(response.text)
    
    # Now we can import it
    from colab_or_local_env import ColabOrLocalEnv
    
# set log level to WARNING
import logging
logging.basicConfig(level=logging.WARNING)

# Initialize our environment
env = ColabOrLocalEnv(
    github_repo="higherbar-ai/open-chat-studio-sim",
    requirements_path="requirements.txt",
    module_paths=["src/ocs_api.py", "src/ocs_simulation_support.py"],
    config_path="~/.ocs/.env",
    config_template={
        "OCS_API_KEY": "",
        "ATHINA_API_KEY": "",
        "EXPERIMENT_ID": "",
        "PARTICIPANT_ID": "open-chat-studio-sim",
    }
)

# Set up the environment (install requirements, fetch modules if needed)
env.setup_environment()

# internal configuration
api_timeout_seconds = 300       # how long to give API calls before timing out
api_num_retries = 3             # how many times to retry API calls before giving up
api_retry_delay_seconds = 2     # how long to wait between retries
continue_on_error = True        # whether to record errors and continue (if False, errors will halt execution)
max_exchanges = 50              # maximum number of user-AI exchanges to simulate in a conversation

# Get API keys from environment
ocs_api_key = env.get_config_setting("OCS_API_KEY")
athina_api_key = env.get_config_setting("ATHINA_API_KEY")
participant_id = env.get_config_setting("PARTICIPANT_ID")
experiment_id = env.get_config_setting("EXPERIMENT_ID")
user_simulator_experiment_id = env.get_config_setting("USER_SIMULATOR_EXPERIMENT_ID")

# Validate required configuration
if not all([ocs_api_key, participant_id, experiment_id]):
    raise ValueError("Please supply at least OCS_API_KEY, PARTICIPANT_ID, and EXPERIMENT_ID in your secrets or configuration file.")

# Output files to ~/ocs directory if local, otherwise /content if Google Colab
if env.is_colab:
    output_path_prefix = "/content"
else:
    import os
    output_path_prefix = os.path.expanduser("~/ocs")
    os.makedirs(output_path_prefix, exist_ok=True)

# initialize OCS and simulation support
from ocs_api import OCSAPIClient    # type: ignore[import]
from ocs_simulation_support import OCSBotToBotSimulator    # type: ignore[import]
ocs_api_client = OCSAPIClient(
    api_key=ocs_api_key, 
    timeout_seconds=api_timeout_seconds, 
    num_retries=api_num_retries, 
    retry_wait_seconds=api_retry_delay_seconds
)
ocs_simulator = OCSBotToBotSimulator(
    ocs_api_client=ocs_api_client, 
    exp_id=experiment_id, 
    user_exp_id=user_simulator_experiment_id, 
    part_id=participant_id
)

# Report results
print(f"Configuration loaded for {'Colab' if env.is_colab else 'local'} environment, OCS API initialized.")

## Select or upload your simulations to run

The code cell below will prompt you to select or upload a .csv file. That file should have the following columns:

- `context`: a description of the context to simulate, which will be sent to the user simulator experiment as the first message (should include everything needed to simulate a user, including background information, why the user is coming to interact with the bot, etc.)
- `simulation_id`: (optional) a unique identifier for the simulation (if not provided, row number will be used)

In [ ]:
# prompt for the CSV file with simulations to run
sims_to_run_files = env.get_input_files("CSV file with simulations to run")

# check for one CSV file
if len(sims_to_run_files) != 1:
    raise ValueError("Please select exactly one CSV file with simulations to run.")
elif not sims_to_run_files[0].endswith(".csv"):
    raise ValueError("Please select a CSV file with simulations to run.")

sims_to_run_file = str(sims_to_run_files[0])

## Execute simulations

The following code block reads a list of simulations to run from the .csv file your selected or uploaded above, executes them, and saves the results to the `simulation_results.csv` file. 

Your .csv file should have the following columns:

- `context`: a description of the context to simulate, which will be sent to the user simulator experiment as the first message (should include everything needed to simulate a user, including background information, why the user is coming to interact with the bot, etc.)
- `simulation_id`: (optional) a unique identifier for the simulation (if not provided, row number will be used)

`simulation_results.csv` will have the following columns:

- `simulation_id`: the unique identifier for the simulation
- `session_id`: the unique identifier for the session
- `context`: the context description
- `query`: the query sent to the AI assistant
- `response`: the response received from the AI assistant

In [ ]:
import csv
import pandas as pd
import os

def simulation_status(status: str, sim_id: str, _simulation_context: str):
    if status == "PRE-SIM":
        print(f"Executing simulation {sim_id}...")


# load input file using pandas
simulations_to_run = pd.read_csv(sims_to_run_file)

# assemble simulations to run
simulations = []
for index, row in simulations_to_run.iterrows():
    # if there's a "simulation_id" column, use that, otherwise use the row number as the simulation ID
    simulation_id = row.get("simulation_id", index+1)
    # add to list of simulations to run
    simulations.append((str(simulation_id), row["context"]))

# execute all the simulations (continuing on error and limiting to 50 exchanges per simulation)
results = ocs_simulator.exec_simulations(simulations, continue_on_error=continue_on_error, max_exchanges=max_exchanges, status_callback=simulation_status)

# save results to output .csv file
output_file = os.path.join(output_path_prefix, "simulation_results.csv")
output_rows = []
with open(output_file, "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["simulation_id", "session_id", "context", "query", "response"], quoting=csv.QUOTE_NONNUMERIC, escapechar='\\')
    writer.writeheader()
    for result in results:
        for query, response in result["messages"]:
            output_row = {
                "simulation_id": result["simulation_id"],
                "session_id": result["experiment_session_id"],
                "context": result["context"],
                "query": query,
                "response": response
            }
            writer.writerow(output_row)
            output_rows.append(output_row)

# report results
print()
print(f"Simulations executed and {len(results)} simulation results saved to {output_file}.")

## Optional: Export results to Athina dataset

If an Athina API key is configured, the results can be exported to an Athina dataset. The dataset will be named `simulations-{experiment_id}-{timestamp}` and will contain the rows from the `simulation_results.csv` file.

In [ ]:
from ocs_simulation_support import athina_create_dataset    # type: ignore[import]

# optionally export the results to an Athina dataset
if athina_api_key:
    # push new dataset to Athina
    dataset_name = f"simulations-{experiment_id}-{pd.Timestamp.now().strftime('%Y%m%d%H%M%S')}"
    dataset_description = f"Simulated conversations for experiment {experiment_id} at {pd.Timestamp.now()}, using user simulator experiment {user_simulator_experiment_id}"
    try:
        dataset = athina_create_dataset(athina_api_key=athina_api_key, dataset_name=dataset_name, dataset_description=dataset_description, dataset_rows=output_rows)
    except Exception as e:
        print(f"Failed to create Athina dataset: {e}")
    else:
        print(f"Results exported to Athina dataset {dataset.id} (name: {dataset_name}).")